## Config and Imports

In [9]:
import os
import pickle
import time
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import (
    precision_recall_fscore_support,
    roc_auc_score,
    accuracy_score,
    precision_recall_curve,
)
from sklearn.metrics import average_precision_score as pr_auc
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt

# If you want Word2Vec for embeddings:
from gensim.models import Word2Vec
import zipfile

# Force a specific random seed for reproducibility (optional)
torch.manual_seed(42)
np.random.seed(42)

class Config:
    """
    Holds hyperparameters, file paths, and general settings.
    In practice, you could store these in a YAML/JSON file.
    """
    # Data paths
    dataset_dir = "../resource"
    # Unzip the file and set the input_file path
    input_file = os.path.join(dataset_dir, "S1_Data.txt")
    with zipfile.ZipFile(os.path.join(dataset_dir, "S1_Data.zip"), 'r') as zip_ref:
        zip_ref.extractall(dataset_dir)
    

    vocab_file = os.path.join(dataset_dir, "vocab.txt")
    stop_file = os.path.join(dataset_dir, "stop.txt")
    vocab_pkl = os.path.join(dataset_dir, "vocab.pkl")

    # PKLs for train, valid, test data
    pkl_train_x = os.path.join(dataset_dir, "X_train.pkl")
    pkl_train_y = os.path.join(dataset_dir, "Y_train.pkl")
    pkl_val_x   = os.path.join(dataset_dir, "X_valid.pkl")
    pkl_val_y   = os.path.join(dataset_dir, "Y_valid.pkl")
    pkl_test_x  = os.path.join(dataset_dir, "X_test.pkl")
    pkl_test_y  = os.path.join(dataset_dir, "Y_test.pkl")

    # For building the vocab
    rare_word_threshold = 100
    stop_word_threshold = 1e4

    unknown_index = 1
    vocab_size = 490
    n_stops = 12  # last 12 are considered "stop words"
    n_topics = 50
    max_visit_len = 300

    # Model hyperparams
    embed_size = 100
    hidden_size = 200
    dropout = 1.0

    # Training settings
    batch_size = 4  # A small batch size example
    grad_clip = 5
    learning_rate = 1e-3
    num_epochs = 2  # Example short run

    # Where to save intermediate outputs
    theta_dir = "theta_with_rnnvec"
    results_dir = "CONTENT_results"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Config] Device: {device}")


FileNotFoundError: [Errno 2] No such file or directory: '../resource/S1_Data.zip'

### EXECUTE Config setup

In [2]:
config = Config()


###  Utility Helpers (Pickle, Numpy, Vocab, etc.)

In [3]:
def save_pkl(path, obj):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
    print(f"Saved: {path}")

def load_pkl(path):
    with open(path, "rb") as f:
        obj = pickle.load(f)
    print(f"Loaded: {path}")
    return obj

def save_npy(path, arr):
    np.save(path, arr)
    print(f"Saved: {path}")

def load_npy(path):
    arr = np.load(path, allow_pickle=True)
    print(f"Loaded: {path}")
    return arr

def build_vocab(config: Config):
    """
    Creates vocab.txt and stop.txt from S1_Data.txt by filtering.
    Index offset so that 'unknown_index' can be used.
    """
    df = pd.read_csv(config.input_file, sep="\t", header=0)
    grouped = df.groupby("DX_GROUP_DESCRIPTION").size().reset_index(name="SIZE")

    # Filter out rare
    grouped = grouped[grouped["SIZE"] > config.rare_word_threshold]

    # Sort by frequency ascending
    grouped = grouped.sort_values(by="SIZE").reset_index(drop=True)
    vocab = grouped["DX_GROUP_DESCRIPTION"]
    vocab.index += 2  # offset => index=1 is reserved for unknown

    vocab.to_csv(config.vocab_file, sep="\t", header=False, index=True)
    print("Number of valid tokens:", len(vocab))

    # Stop words => extremely frequent
    stops = grouped[grouped["SIZE"] > config.stop_word_threshold]
    stops["DX_GROUP_DESCRIPTION"].to_csv(config.stop_file, sep="\t", header=False, index=False)

def load_vocab_dict(config: Config):
    """
    Reads vocab_file => returns {word: index}, also pickles reverse mapping.
    """
    word_to_index = {}
    with open(config.vocab_file, "r") as f:
        for line in f:
            idx_str, token = line.strip().split("\t")
            word_to_index[token] = int(idx_str) - 1

    reverse_mapping = {v: k for k, v in word_to_index.items()}
    save_pkl(config.vocab_pkl, reverse_mapping)
    print(f"Vocab size: {len(word_to_index)}")
    return word_to_index


### Data Preprocessing and Splitting

In [4]:
def extract_inpatient_events(config: Config):
    """
    Extracts 'INPATIENT HOSPITAL' events => used to mark readmission (1) if next event is within 30 days.
    """
    df = pd.read_csv(config.input_file, sep="\t", header=0)
    inpat = df[df["SERVICE_LOCATION"] == "INPATIENT HOSPITAL"]
    grouped = (inpat.groupby(["PID", "DAY_ID", "SERVICE_LOCATION"])
                    .size()
                    .reset_index(name="COUNT")
                    .sort_values(["PID", "DAY_ID"], ascending=True)
                    .set_index("PID"))
    return grouped

def convert_format(config: Config, word2idx, inpat_df):
    """
    Goes through S1_Data.txt line by line => builds docs & labels.
    docs[i] = [ [codes_on_day1], [codes_on_day2], ... ]
    labels[i] = [0/1 for each day]
    """
    def is_readmitted(pid, day):
        try:
            recs = inpat_df.loc[int(pid)]
            if isinstance(recs, pd.Series):
                return (int(day) <= recs.DAY_ID < int(day) + 30)
            # else a sub-DataFrame
            subset = recs.loc[(int(day) <= recs.DAY_ID) & (recs.DAY_ID < int(day) + 30)]
            return subset.shape[0] > 0
        except KeyError:
            return False

    docs, labels = [], []
    with open(config.input_file, "r") as f:
        header = f.readline().strip().split("\t")
        col_idx = {h: i for i, h in enumerate(header)}

        doc, visit_codes, label_seq = [], [], []
        line = f.readline()
        if not line:
            return docs, labels

        tokens = line.strip().split("\t")
        pid, day_id = tokens[col_idx["PID"]], tokens[col_idx["DAY_ID"]]
        label_seq.append(1 if is_readmitted(pid, day_id) else 0)

        while line:
            tokens = line.strip().split("\t")
            c_pid, c_day = tokens[col_idx["PID"]], tokens[col_idx["DAY_ID"]]

            if c_pid != pid:
                doc.append(visit_codes)
                docs.append(doc)
                labels.append(label_seq)
                # reset
                doc, visit_codes, label_seq = [], [], []
                pid, day_id = c_pid, c_day
                label_seq.append(1 if is_readmitted(pid, day_id) else 0)
            else:
                # same patient, check if new day
                if c_day != day_id:
                    doc.append(visit_codes)
                    visit_codes = []
                    day_id = c_day
                    label_seq.append(1 if is_readmitted(pid, day_id) else 0)

            diag_str = tokens[col_idx["DX_GROUP_DESCRIPTION"]]
            diag_idx = word2idx.get(diag_str, config.unknown_index)
            visit_codes.append(diag_idx)
            line = f.readline()

        # finalize
        doc.append(visit_codes)
        docs.append(doc)
        labels.append(label_seq)

    return docs, labels

def split_and_save(config: Config, docs, labels):
    """
    Splits docs/labels => train/valid/test and saves as pkl.
    """
    # adjust these splits as needed
    train_end = 2000
    val_end = 2500

    save_pkl(config.pkl_train_x, docs[:train_end])
    save_pkl(config.pkl_train_y, labels[:train_end])

    save_pkl(config.pkl_val_x, docs[train_end:val_end])
    save_pkl(config.pkl_val_y, labels[train_end:val_end])

    save_pkl(config.pkl_test_x, docs[val_end:])
    save_pkl(config.pkl_test_y, labels[val_end:])


### PyTorch Dataset & DataLoader

In [5]:
class PatientVisitsDataset(Dataset):
    """
    A PyTorch Dataset that holds (docs, labels) for a split (train/valid/test).
    We'll convert them to multi-hot within __getitem__ or in a collate_fn.
    """
    def __init__(self, docs, labels, vocab_size, max_len):
        super().__init__()
        self.docs = docs
        self.labels = labels
        self.vocab_size = vocab_size
        self.max_len = max_len

    def __len__(self):
        return len(self.docs)

    def __getitem__(self, idx):
        return self.docs[idx], self.labels[idx]

def multi_hot_collate_fn(batch, vocab_size, max_len):
    """
    Collate function to transform a list of (doc, label) => (x, y, mask).
    Each doc is a list of visits.
    We'll multi-hot each visit, up to max_len visits.
    """
    # We'll first figure out batch_size from the list
    batch_size = len(batch)

    # 1) separate docs and labels
    docs = [b[0] for b in batch]
    labels = [b[1] for b in batch]

    # 2) find the truncated length
    truncated_docs = []
    truncated_labels = []
    lengths = []
    for doc, lab in zip(docs, labels):
        if len(doc) <= max_len:
            truncated_docs.append(doc)
            truncated_labels.append(lab)
            lengths.append(len(doc))
        else:
            truncated_docs.append(doc[:max_len])
            truncated_labels.append(lab[:max_len])
            lengths.append(max_len)

    max_len_batch = max(lengths) if lengths else 0
    # 3) Allocate arrays
    x_array = np.zeros((batch_size, max_len_batch, vocab_size), dtype=np.float32)
    y_array = np.ones((batch_size, max_len_batch), dtype=np.float32)
    mask_array = np.zeros((batch_size, max_len_batch), dtype=np.float32)

    # 4) Fill
    for i, (doc, lab) in enumerate(zip(truncated_docs, truncated_labels)):
        seq_len = len(doc)
        mask_array[i, :seq_len] = 1
        y_array[i, :seq_len] = lab
        for j, visit_codes in enumerate(doc):
            for code_idx in visit_codes:
                x_array[i, j, code_idx-1] = 1

    # 5) Convert to torch
    x_tensor = torch.from_numpy(x_array)
    y_tensor = torch.from_numpy(y_array)
    mask_tensor = torch.from_numpy(mask_array)
    return x_tensor, y_tensor, mask_tensor

def create_dataloader(docs, labels, config: Config, shuffle=False):
    """
    Convenience method to build a DataLoader from docs/labels.
    """
    dataset = PatientVisitsDataset(docs, labels, config.vocab_size, config.max_visit_len)
    loader = DataLoader(
        dataset,
        batch_size=config.batch_size,
        shuffle=shuffle,
        collate_fn=lambda b: multi_hot_collate_fn(b, config.vocab_size, config.max_visit_len),
    )
    return loader


### EXECUTE Data Prep

In [6]:
# 1) Build vocab & Load it
build_vocab(config)
w2i = load_vocab_dict(config)

# 2) Extract events => docs => split => pkl
inpat_events = extract_inpatient_events(config)
docs, labels = convert_format(config, w2i, inpat_events)
split_and_save(config, docs, labels)

# 3) Load those splits into memory
X_train = load_pkl(config.pkl_train_x)
Y_train = load_pkl(config.pkl_train_y)

X_valid = load_pkl(config.pkl_val_x)
Y_valid = load_pkl(config.pkl_val_y)

X_test  = load_pkl(config.pkl_test_x)
Y_test  = load_pkl(config.pkl_test_y)

# 4) Create DataLoaders
train_loader = create_dataloader(X_train, Y_train, config, shuffle=True)
valid_loader = create_dataloader(X_valid, Y_valid, config, shuffle=False)
test_loader  = create_dataloader(X_test,  Y_test,  config, shuffle=False)


FileNotFoundError: [Errno 2] No such file or directory: '../resource/S1_Data.txt'

### Defining CONTENT Model

In [ ]:
class ThetaLayer(nn.Module):
    """
    Reparameterization for the topic distribution + KL term.
    """
    def __init__(self, max_len, n_topics):
        super().__init__()
        self.max_len = max_len
        self.klterm = 0.0
        self.theta = None
        self.n_topics = n_topics

    def forward(self, mu, log_sigma):
        eps = torch.randn_like(mu)
        z = mu + torch.exp(0.5 * log_sigma) * eps
        theta = F.softmax(z, dim=1)
        self.theta = theta

        # kl
        self.klterm = -0.5 * torch.sum(1 + log_sigma - mu.pow(2) - log_sigma.exp())

        # Expand => [batch_size, seq_len, n_topics]
        expanded_theta = theta.unsqueeze(1).expand(-1, self.max_len, self.n_topics)
        return expanded_theta


class ContentModel(nn.Module):
    """
    RNN + Topic model:
      1) embed => GRU
      2) dense => mu, log_sigma => Theta
      3) B * Theta => context
      4) sum => final prediction
    """
    def __init__(self, config: Config):
        super().__init__()
        self.vocab_size = config.vocab_size
        self.embed_size = config.embed_size
        self.hidden_size= config.hidden_size
        self.n_topics   = config.n_topics
        self.max_len    = config.max_visit_len

        self.embed = nn.Linear(self.vocab_size, self.embed_size, bias=False)
        self.gru = nn.GRU(self.embed_size, self.hidden_size, batch_first=True)

        self.dense1 = nn.Linear(self.vocab_size, self.hidden_size)
        self.dense2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.mu = nn.Linear(self.hidden_size, self.n_topics)
        self.log_sigma = nn.Linear(self.hidden_size, self.n_topics)

        self.B = nn.Linear(self.vocab_size, self.n_topics, bias=False)
        self.out_layer = nn.Linear(self.hidden_size, 1)

        self.theta_layer = ThetaLayer(self.max_len, self.n_topics)

    def forward(self, x, mask):
        # x => [batch, seq_len, vocab_size]
        embedded = self.embed(x) * mask.unsqueeze(-1)  # [B, T, embed_size]
        gru_out, h_n = self.gru(embedded)

        # topic
        h1 = F.relu(self.dense1(x))  # [B, T, hidden]
        h2 = F.relu(self.dense2(h1)) # [B, T, hidden]

        avg_h2 = h2.mean(dim=1)      # [B, hidden]
        mu_val = self.mu(avg_h2)
        log_sigma_val = self.log_sigma(avg_h2)

        theta_expanded = self.theta_layer(mu_val, log_sigma_val)  # [B, T, n_topics]

        # context
        B_out = self.B(x)  # [B, T, n_topics]
        context = (B_out * theta_expanded).mean(dim=-1)  # [B, T]

        # combine
        rnn_scores = self.out_layer(gru_out).squeeze(-1)  # [B, T]
        logits = rnn_scores + context
        out = torch.sigmoid(logits)
        out = out * mask + 1e-6  # mask out
        return out, h_n, self.theta_layer.theta

    @property
    def kl_term(self):
        return self.theta_layer.klterm


### EXECUTE Model Initation

In [ ]:
# 5) Build Model
model = ContentModel(config).to(config.device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)


### Train Model

In [ ]:
def train(model, loader, optimizer, config: Config):
    """
    Train step over 'loader' => returns average train loss, plus any collected [theta+hidden].
    """
    model.train()
    total_loss = 0.
    batch_count= 0
    collector  = []

    for x_batch, y_batch, m_batch in loader:
        x_batch = x_batch.to(config.device)
        y_batch = y_batch.to(config.device)
        m_batch = m_batch.to(config.device)

        optimizer.zero_grad()
        preds, h_n, theta = model(x_batch, m_batch)

        bce = F.binary_cross_entropy(preds.view(-1), y_batch.view(-1), reduction="sum")
        loss= bce + model.kl_term
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip)
        optimizer.step()

        total_loss += loss.item()
        batch_count+= 1

        # store [theta + hidden]
        rnn_vec = h_n.squeeze(0).detach().cpu().numpy()
        theta_np= theta.detach().cpu().numpy()
        combined= np.concatenate([theta_np, rnn_vec], axis=1)
        collector.append(combined)

    avg_loss = total_loss / max(batch_count,1)
    return avg_loss, collector


### EXECUTE Training

In [ ]:
for epoch in range(config.num_epochs):
  st = time.time()

  # --- Train on train set ---
  train_loss, train_theta_collector = train(model, train_loader, optimizer, config)
  train_thetas_arr = np.concatenate(train_theta_collector, axis=0)
  np.save(os.path.join(config.theta_dir, f"thetas_train_{epoch}.npy"), train_thetas_arr)

  elapsed = time.time() - st
  print(f"\nEpoch {epoch+1}/{config.num_epochs} took {elapsed:.2f}s")
  print(f"  [Train] loss={train_loss:.4f}")


### Testing Model

In [ ]:
def evaluate_model(model, loader, config: Config):
    """
    Evaluates on a DataLoader (e.g. test/valid).
    Returns (avg_loss, list_of_true, list_of_pred, theta_hidden_collector).
    """
    model.eval()
    total_loss = 0.0
    batch_count= 0
    all_true, all_pred = [], []
    all_theta_hidden = []

    with torch.no_grad():
        for x_batch, y_batch, mask_batch in loader:
            x_batch = x_batch.to(config.device)
            y_batch = y_batch.to(config.device)
            mask_batch = mask_batch.to(config.device)

            preds, h_n, theta = model(x_batch, mask_batch)
            bce = F.binary_cross_entropy(preds.view(-1), y_batch.view(-1), reduction="sum")
            loss= bce + model.kl_term

            total_loss += loss.item()
            batch_count += 1

            # flatten predictions/labels ignoring masked positions
            seq_lens = mask_batch.sum(dim=1).cpu().numpy().astype(int)
            preds_np = preds.detach().cpu().numpy()
            y_np     = y_batch.detach().cpu().numpy()

            for i in range(x_batch.shape[0]):
                length_i = seq_lens[i]
                all_pred.extend(preds_np[i, :length_i])
                all_true.extend(y_np[i, :length_i])

            # store [theta + hidden]
            rnn_vec = h_n.squeeze(0).detach().cpu().numpy()
            theta_np= theta.detach().cpu().numpy()
            combined= np.concatenate([theta_np, rnn_vec], axis=1)
            all_theta_hidden.append(combined)

    avg_loss = total_loss / (batch_count if batch_count else 1)
    return avg_loss, all_true, all_pred, all_theta_hidden


def compute_metrics(true_vals, pred_vals):
    """
    Returns a dict with AUC, PR-AUC, ACC, Precision, Recall, F1.
    """
    auc_val = roc_auc_score(true_vals, pred_vals)
    pr_val  = pr_auc(true_vals, pred_vals)
    preds_bin = (np.array(pred_vals) > 0.5).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(true_vals, preds_bin, average="binary")
    acc_val = accuracy_score(true_vals, preds_bin)
    return {
        "auc": auc_val,
        "prauc": pr_val,
        "acc": acc_val,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }


### EXECUTE Testing

In [ ]:
# Step F: final test (completely separate from training loop)
test_loss, test_true, test_pred, test_theta_collector = evaluate_model(model, test_loader, config)
test_metrics = compute_metrics(test_true, test_pred)

# --- Validate on val set (optional each epoch) ---
val_loss, val_true, val_pred, val_theta_collector = evaluate_model(model, valid_loader, config)
val_metrics = compute_metrics(val_true, val_pred)

# Save the test results
test_thetas_arr = np.concatenate(test_theta_collector, axis=0)
np.save(os.path.join(config.theta_dir,  f"thetas_test_final.npy"), test_thetas_arr)
np.save(os.path.join(config.results_dir,f"test_labels_final.npy"), np.array(test_true))
np.save(os.path.join(config.results_dir,f"test_preds_final.npy"),  np.array(test_pred))

print(f"[Test] loss={test_loss:.4f}, AUC={test_metrics['auc']:.4f}, "
      f"PR-AUC={test_metrics['prauc']:.4f}, ACC={test_metrics['acc']:.4f}, "
      f"Precision={test_metrics['precision']:.4f}, Recall={test_metrics['recall']:.4f}, "
      f"F1={test_metrics['f1']:.4f}")

print(f"[Valid] loss={val_loss:.4f}, AUC={val_metrics['auc']:.4f}, PR-AUC={val_metrics['prauc']:.4f}, "
        f"ACC={val_metrics['acc']:.4f}, Precision={val_metrics['precision']:.4f}, "
        f"Recall={val_metrics['recall']:.4f}, F1={val_metrics['f1']:.4f}")
